In [1]:
import torch
from torch import nn
from torch import functional as F
from torch import optim 
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import numpy as np
import time
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# TSP data

In [3]:
import itertools
np.random.seed(0)

def tsp_opt(points):
    """
    Dynamic programing solution for TSP - O(2^n*n^2)
    https://gist.github.com/mlalevic/6222750
    :param points: List of (x, y) points
    :return: Optimal solution
    """

    def length(x_coord, y_coord):
        return np.linalg.norm(np.asarray(x_coord) - np.asarray(y_coord))

    # Calculate all lengths
    all_distances = [[length(x, y) for y in points] for x in points]
    # Initial value - just distance from 0 to every other point + keep the track of edges
    A = {(frozenset([0, idx+1]), idx+1): (dist, [0, idx+1]) for idx, dist in enumerate(all_distances[0][1:])}
    cnt = len(points)
    for m in range(2, cnt):
        B = {}
        for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, cnt), m)]:
            for j in S - {0}:
                # This will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
                B[(S, j)] = min([(A[(S-{j}, k)][0] + all_distances[k][j], A[(S-{j}, k)][1] + [j])
                                 for k in S if k != 0 and k != j])
        A = B
    res = min([(A[d][0] + all_distances[0][d[1]], A[d][1]) for d in iter(A)])
    res[1].append(res[1][0])
    return np.asarray(res[1])

In [4]:
np.random.seed(0)

NUM_SAMPLES = 100000
MAX_LENGTH = 7
MIN_LENGTH = 7

lengths = np.random.randint(MIN_LENGTH, MAX_LENGTH + 1, size=(NUM_SAMPLES))
coordinates = [np.random.uniform(size=[l,2]) for l in lengths]
solutions = [tsp_opt(points) for points in tqdm(coordinates)]

100%|██████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [04:27<00:00, 374.04it/s]


In [5]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(coordinates, solutions, test_size=0.7)

In [6]:
class TSPData(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]).type(torch.FloatTensor).to(device), \
    torch.from_numpy(self.y[idx]).type(torch.LongTensor).to(device)
    
    def __len__(self):
        return len(self.X)

In [7]:
train_dataset = TSPData(input_tensor_train, target_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=16)

In [8]:
val_dataset = TSPData(input_tensor_val, target_tensor_val)
val_loader = DataLoader(val_dataset, batch_size=1)

In [9]:
it = iter(train_loader)
for x,y in it:
    print(x,y)
    break

tensor([[[0.5770, 0.9926],
         [0.2732, 0.5652],
         [0.7807, 0.2254],
         [0.5788, 0.3082],
         [0.0294, 0.4081],
         [0.3703, 0.6464],
         [0.5723, 0.9969]],

        [[0.4787, 0.9719],
         [0.6227, 0.2497],
         [0.6690, 0.0903],
         [0.7040, 0.7991],
         [0.5859, 0.2175],
         [0.0724, 0.0400],
         [0.7533, 0.4558]],

        [[0.4553, 0.0151],
         [0.6435, 0.0592],
         [0.0685, 0.9878],
         [0.0184, 0.8300],
         [0.2522, 0.7085],
         [0.4245, 0.0247],
         [0.1513, 0.9368]],

        [[0.4697, 0.6913],
         [0.8001, 0.7468],
         [0.3714, 0.7552],
         [0.9143, 0.3212],
         [0.0629, 0.4416],
         [0.6047, 0.3109],
         [0.6033, 0.9090]],

        [[0.3453, 0.9383],
         [0.6550, 0.0704],
         [0.3712, 0.3764],
         [0.4133, 0.0650],
         [0.2065, 0.5442],
         [0.3056, 0.0057],
         [0.1013, 0.9888]],

        [[0.8987, 0.0128],
         [0.0792, 

In [10]:
x.size(), y.size()

(torch.Size([16, 7, 2]), torch.Size([16, 8]))

# Encoder

In [11]:
class Encoder(nn.Module):
    
    def __init__(self, encoder_dim, embedding_dim, is_bidir):
        super().__init__()
        self.encoder_dim = encoder_dim
        self.embedding_dim = embedding_dim
        self.is_bidir = is_bidir
        self.embedding = nn.Linear(2, self.embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, encoder_dim, bidirectional = is_bidir, batch_first = True)
        
    def forward(self, inputs):
        b_sz = inputs.size(0)
        x = self.embedding(inputs)
        self.h_0, self.c_0 = self.init_hidden(b_sz), self.init_hidden(b_sz)
        outputs, states = self.lstm(x, (self.h_0, self.c_0))
        return outputs, states  #make batch_first
        
    def init_hidden(self, b_sz):
        D = 2 if self.is_bidir else 1
        return torch.zeros(D, b_sz, self.encoder_dim).to(device)

In [12]:
UNITS = 256
EMBEDS = 128

In [13]:
encoder = Encoder(encoder_dim = UNITS, 
                  embedding_dim = EMBEDS, 
                  is_bidir = True).to(device)

In [14]:
class Decoder(nn.Module):
    def __init__(self, decoder_dim, encoder_dim, embedding_dim, is_enc_bidir = True):
        
        super().__init__()
        self.decoder_dim = 2*encoder_dim if is_enc_bidir else encoder_dim
        self.encoder_dim = encoder_dim
        self.embedding_dim = embedding_dim
        
        self.embedding = nn.Linear(2, self.embedding_dim)
        self.lstm = nn.LSTM(self.embedding_dim, self.decoder_dim, batch_first = True)
              
        W1_indim = 2*self.encoder_dim if is_enc_bidir == True else self.encoder_dim
        self.W1 = nn.Linear(W1_indim, self.decoder_dim, bias=False) 
        self.W2 = nn.Linear(self.decoder_dim, self.decoder_dim, bias=False)
        self.V = nn.Linear(self.decoder_dim, 1, bias=False)
        
    def forward(self, x, states, encoder_outputs):
        "x: [1, 2]: coordinate for point 1, batched"
        "prev_hidden: encoder_hidden"
        b_sz = x.size(0)
        x = self.embedding(x)
        _, dec_states = self.lstm(x, states)
        
        score = torch.tanh(self.W1(encoder_outputs) + self.W2(dec_states[0].permute(1,0,2)))
        
        output_probs = self.V(score).view(b_sz, -1)
        
        return output_probs, dec_states

In [15]:
decoder = Decoder(decoder_dim = UNITS, 
                  encoder_dim = UNITS, 
                  embedding_dim = EMBEDS).to(device)

# Training

In [16]:
loss_fn = nn.CrossEntropyLoss()

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()),
                       lr=0.001)

In [ ]:
EPOCHS = 50
loss_epochs = []

for epoch in tqdm(range(EPOCHS), position=0, desc='EPOCHS'):
    total_loss = 0
    
    for X, y in train_loader:
    
        b_sz = X.size(0)
        loss = 0
        enc_hidden_all, dec_hidden = encoder(X)
        dec_hidden = [x.permute(1,0,2).reshape(1, b_sz, -1) for x in dec_hidden]
        dec_input = X[:, 0].unsqueeze(1)

        for t in range(1, y.size(1)):
            predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_hidden_all)
            loss += loss_fn(predictions, y[:, 1])
            if np.random.rand() > 0.999:
                #use predicted best
                pred_token = torch.argmax(torch.nn.functional.softmax(predictions, dim=-1), dim=-1)
                dec_input = X[torch.arange(b_sz), pred_token, :].unsqueeze(1)
            else:
                dec_input = X[torch.arange(b_sz), y.squeeze()[:, t], :].unsqueeze(1) # teacher forcing

        batch_loss = (loss.item() / int(y.size(1)))
        total_loss += batch_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss_epochs.append(total_loss)

In [ ]:
from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME_1 = "encoder.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME_1

torch.save(obj=encoder.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

MODEL_NAME_2 = "decoder.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME_2

torch.save(obj=decoder.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

In [ ]:
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME_1
encoder.load_state_dict(torch.load(f = MODEL_SAVE_PATH))

In [ ]:
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME_2
decoder.load_state_dict(torch.load(f = MODEL_SAVE_PATH))

In [ ]:
X, y

In [ ]:
for X, y in val_loader:
    with torch.no_grad():
        b_sz = X.size(0)
        enc_hidden_all, enc_hidden_last = encoder(X)
        dec_hidden = [x.permute(1,0,2).reshape(1, b_sz, -1) for x in enc_hidden_last]
        dec_input = X[:, 0].unsqueeze(1)
        tour = []
        pred_token = None
        while pred_token != 0:
            predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_hidden_all)
            pred_token = torch.argmax(torch.nn.functional.softmax(predictions, dim=-1), dim=-1)
            tour.append(pred_token.item())
            print(pred_token.item())
    print(y)
    break

In [ ]:
dec_input.shape, dec_hidden[0].shape, dec_hidden[1].shape, enc_hidden_all.shape